In [ ]:
pattern = r'([A-Za-z\s]+)\s*\(Managing Since\s*([A-Za-z0-9\s]+) and overall experience of ([a-z0-9\s]+)\)'
nav_pattern = r'((?:Regular|Direct|Reg)\s*(?:Growth|IDCW))\s*([\d,.]+)'
pattern = r'(Std. Dev|Sharpe Ratio|Portfolio Beta|R Squared|Treynor|Jenson)\s+([\d.-]+|NA)\s+([\d.-]+|NA)'
expense_pattern = r'(Regular|Direct)\s*([\d,.]+)'
ptr_pattern = r'([\d,]+\.\d+)'
for fund, content in final_text.items():
    check = 'portfolio_turnover_ratio'
    if check in content:
        # all_text = " ".join(content[check])
        # all_text = re.sub(r'[\^\-,:]+'," ", all_text.strip())
        for text in content[check]:
            text = re.sub(r'[\^\-,:]+'," ", text.strip())
            if matches := re.findall(ptr_pattern, text, re.IGNORECASE):
                print(matches)
        # for text in content[check]:
        #     print(text)

In [ ]:
for fund, content in final_text.items():
    check = 'fund_manager'
    if check in content:
        all_text = " ".join(content[check])
        all_text =re.sub(r"[\^*\$\*\(\):\.]", "", all_text.strip())
        print(all_text)

In [ ]:
[' ',
 ' years v',
 ' yeavrs',
 ' yrs v',
 '#note:',
 '(annualised)',
 '(directional_equity):',
 '(earlier known as Taurus Largecap Equity Fund)',
 '(erstwhile known as DSP Equal Nif',
 '(last_0_months):',
 '(last_10_months):',
 '(last_11_months)',
 '(last_11_months):',
 '(last_12_months)',
 '(last_3_months):',
 '(last_7_months):',
 '(Maturity of 1 day)',
 '(Maturity of upto 91 days)',
 '(Maturity upto 1 year)',
 '(Number of Segregated porolio in the scheme 1)',
 '(Number of Segregated portfolio in the scheme 1)',
 '(Number of Segregated portfolio in the scheme 3)',
 '(Number of Segregated portfolio in the scheme 4)',
 '*aum',
 '.0278',
 '.3008',
 '.4521',
 '.9577_rating_profile_of_portfolio',
 '@',
 '_refer_page_no_59',
 '_to_nav',
 '0',
 '0076',
 '024)',
 '0599',
 '0600',
 '1',
 '1_year',
 '1_year_3',
 '1_year_3_y',
 '1_year_3_ye',
 '1_year_3_years',
 '1_year_3_years_5',
 '100.00%',
 '112',
 '12',
 '15d_3y',
 '1jul11',
 '1m_5y',
 '1y',
 '2',
 '20may09',
 '2116',
 '26aug09',
 '2763',
 '27nov24',
 '2808_rating_profile_of_portfolio',
 '2989',
 '3_year_risk_statistics:',
 '30dec24',
 '3541',
 '360 one balanced hybrid fund',
 '360 one dynamic bond fund',
 '360 one elss tax saver nifty 50 index fund',
 '360 one flexicap fund',
 '360 one focused equity fund',
 '360 one liquid fund',
 '360 one quant fund',
 '3y',
 '4',
 '4)',
 '4077',
 '443',
 '5',
 '500_thereafter',
 '5006',
 '5mar20',
 '5y',
 '6088',
 '6294',
 '6426',
 '6800_rating_profile_of_portfolio',
 '7',
 '7_days_15_days_30_days_1_year_3_y',
 '7000',
 '7074_rating_profile_of_portfolio',
 '7206_rating_profile_of_portfolio',
 '7aug13',
 '7d_1y',
 '7feb14',
 '7jul14',
 '8',
 '8518',
 '861',
 '8830',
 '8902_rating_profile_of_portfolio',
 '9100_nav_of_plans_/_options_(_)',
 '9539',
 '9688',
 '9873',
 'aaum',
 'AAUM',
 'acking_differences_regular_direct',
 'Active Stock Position in Scheme',
 'Additional',
 'Additional Benchmark',
 'Additional Investment Amount',
 'Additional Purchase Amount',
 'additional_benchmark:',
 'additional_investment',
 'additional_investment_amount',
 'additional_purchase',
 'additional_purchase_amount',
 'aditya birla sun life arbitrage fund',
 'aditya birla sun life asset allocator fof',
 'aditya birla sun life balanced advantage fund',
 'aditya birla sun life banking and financial services fund',
 'aditya birla sun life bse sensex etf',
 'aditya birla sun life credit risk fund',
 'aditya birla sun life crisil 10 year gilt etf',
 'aditya birla sun life crisil broad based gilt etf',
 'aditya birla sun life crisil ibx 50:50 gilt plus sdl apr 2028 index fund',
 'aditya birla sun life crisil ibx 60:40 sdl + aaa psu apr 2026 index fund',
 'aditya birla sun life crisil ibx gilt - april 2026 index fund',
 'aditya birla sun life crisil ibx gilt apr 2028 index fund',
 'aditya birla sun life crisil ibx gilt apr 2029 index fund',
 'aditya birla sun life crisil ibx gilt april 2033 index fund',
 'aditya birla sun life crisil ibx gilt june 2027 index fund',
 'aditya birla sun life crisil ibx sdl jun 2032 index fund',
 'aditya birla sun life crisil liquid overnight etf',
 'aditya birla sun life crisil-ibx aaa financial services index - sep 2027 fund',
 'aditya birla sun life crisil-ibx aaa nbfc-hfc index – dec 2025 fund',
 'aditya birla sun life crisil-ibx aaa nbfc-hfc index – sep 2026 fund',
 'aditya birla sun life crisil-ibx financial services 3 to 6 months debt index fund',
 'aditya birla sun life digital india fund',
 'aditya birla sun life dividend yield fund',
 'aditya birla sun life dynamic bond fund',
 'aditya birla sun life elss tax saver fund',
 'aditya birla sun life equity advantage fund',
 'aditya birla sun life equity hybrid ‘95 fund',
 'aditya birla sun life equity savings fund',
 'aditya birla sun life esg integration strategy fund',
 'aditya birla sun life financial planning fof - aggressive plan',
 'aditya birla sun life financial planning fof - conservative plan',
 'aditya birla sun life financial planning fof - moderate plan',
 'aditya birla sun life flexi cap fund',
 'aditya birla sun life focused fund',
 'aditya birla sun life frontline equity fund',
 'aditya birla sun life global emerging opportunities fund',
 'aditya birla sun life global excellence equity fund of fund',
 'aditya birla sun life gold etf',
 'aditya birla sun life gold fund',
 'aditya birla sun life government securities fund',
 'aditya birla sun life india gennext fund',
 'aditya birla sun life infrastructure fund',
 'aditya birla sun life liquid fund',
 'aditya birla sun life long duration fund',
 'aditya birla sun life low duration fund',
 'aditya birla sun life manufacturing equity fund',
 'aditya birla sun life midcap fund',
 'aditya birla sun life mnc fund',
 'aditya birla sun life multi - index fund of funds',
 'aditya birla sun life multi asset allocation fund',
 'aditya birla sun life multi-cap fund',
 'aditya birla sun life nifty 200 momentum 30 etf',
 'aditya birla sun life nifty 200 quality 30 etf',
 'aditya birla sun life nifty 50 equal weight index fund',
 'aditya birla sun life nifty 50 etf',
 'aditya birla sun life nifty 50 index fund',
 'aditya birla sun life nifty bank etf',
 'aditya birla sun life nifty healthcare etf',
 'aditya birla sun life nifty it etf',
 'aditya birla sun life nifty midcap 150 index fund',
 'aditya birla sun life nifty next 50 etf',
 'aditya birla sun life nifty pse etf',
 'aditya birla sun life nifty sdl sep 2025 index fund',
 'aditya birla sun life nifty sdl sep 2027 index fund',
 'aditya birla sun life nifty smallcap 50 index fund',
 'aditya birla sun life overnight fund',
 'aditya birla sun life pharma & healthcare fund',
 'aditya birla sun life psu equity fund',
 'aditya birla sun life pure value fund',
 'aditya birla sun life regular savings fund',
 'aditya birla sun life retirement fund - the 30s plan',
 'aditya birla sun life retirement fund - the 40s plan',
 'aditya birla sun life short term fund',
 'aditya birla sun life silver etf',
 'aditya birla sun life silver etf fund of fund',
 'aditya birla sun life small cap fund',
 'aditya birla sun life special opportunities fund',
 'aditya birla sun life transportation and logistics fund',
 'Aggregate',
 'aggregate_value_of_investments_by_other_schemes_of_bank_of_india_mutual_fund_as_on_31_december_2024_is_297.67_lacs.',
 'aggregate_value_of_investments_by_other_schemes_of_bank_of_india_mutual_fund_as_on_31_december_2024_is_6651.21_lacs.',
 'aggressive_hybrid_fund',
 'Allotment Date',
 'allotment_date',
 'Also manages',
 'Annualised Portfo!!lio YTM',
 'Annualised Portfo!l!io YTM',
 'Annualised Portfolio YTM',
 'Annualised Yield',
 'Annualised YTM',
 'annualised_portfolio_ytm',
 'Annualized Portfolio YTM',
 'annualized_portfolio_ytm*',
 'applicable',
 'application_amount_for_fresh_subscription',
 'arbitrage_fund',
 'arbitrage_investments',
 'ASSET ALLOCATION',
 'asset_allocation',
 'Assets Under Management',
 'assistant_fund_manager',
 'aum',
 'AUM',
 'Average / Residual Maturity',
 'Average AUM',
 'Average Maturity',
 'average maturity',
 'Average P/B',
 'Average P/E',
 'average_aum',
 'average_aum#',
 'average_maturity',
 'Avg. AUM',
 'bajaj finserv arbitrage fund',
 'bajaj finserv balanced advantage fund',
 'bajaj finserv banking and psu fund',
 'bajaj finserv consumption fund',
 'bajaj finserv flexi cap fund',
 'bajaj finserv healthcare fund',
 'bajaj finserv large and mid cap fund',
 'bajaj finserv large cap fund',
 'bajaj finserv liquid fund',
 'bajaj finserv money market fund',
 'bajaj finserv multi asset allocation fund',
 'bajaj finserv nifty 50 etf',
 'bajaj finserv nifty bank etf',
 'bajaj finserv overnight fund',
 'Balanced Advantage Fund',
 'bandhan all seasons bond fund',
 'bandhan arbitrage fund',
 'bandhan asset allocation fund of funds',
 'bandhan balanced advantage fund',
 'bandhan banking & psu debt fund',
 'bandhan bond fund – income plan',
 'bandhan bond fund – medium term plan',
 'bandhan bond fund – short term plan',
 'bandhan bse healthcare index fund',
 'bandhan bse sensex etf',
 'bandhan business cycle fund',
 'bandhan conservative hybrid fund',
 'bandhan core equity fund',
 'bandhan corporate bond fund',
 'bandhan credit risk fund',
 'bandhan crisil ibx 90:10 sdl plus gilt– november 2026 index fund',
 'bandhan crisil ibx 90:10 sdl plus gilt– september 2027 index fund',
 'bandhan crisil ibx gilt april 2026 index fund',
 'bandhan crisil ibx gilt april 2028 index fund',
 'bandhan crisil ibx gilt april 2032 index fund',
 'bandhan crisil ibx gilt june 2027 index fund',
 'bandhan dynamic bond fund',
 'bandhan elss tax saver fund',
 'bandhan equity savings fund',
 'bandhan financial services fund',
 'bandhan flexi cap fund',
 'bandhan floating rate fund',
 'bandhan focused equity fund',
 'bandhan government securities fund - constant maturity plan',
 'bandhan government securities fund – investment plan',
 'bandhan hybrid equity fund',
 'bandhan infrastructure fund',
 'bandhan innovation fund',
 'bandhan large cap fund',
 'bandhan liquid fund',
 'bandhan long duration fund',
 'bandhan low duration fund',
 'bandhan midcap fund',
 'bandhan money manager fund',
 'bandhan multi asset allocation fund',
 'bandhan multi cap fund',
 'bandhan nifty 100 index fund',
 'bandhan nifty 200 quality 30 index fund',
 'bandhan nifty 50 etf',
 'bandhan nifty 50 index fund',
 'bandhan nifty 500 momentum 50 index fund',
 'bandhan nifty 500 value 50 index fund',
 'bandhan nifty alpha 50 index fund',
 'bandhan nifty bank index fund',
 'bandhan nifty it index fund',
 'bandhan nifty midcap 150 index fund',
 'bandhan nifty smallcap 250 index fund',
 'bandhan nifty total market index fund',
 'bandhan nifty100 low volatility 30 index fund',
 'bandhan nifty200 momentum 30 index fund',
 'bandhan overnight fund',
 'bandhan retirement fund',
 'bandhan small cap fund',
 'bandhan sterling value fund',
 'bandhan transportation and logistics fund',
 'bandhan ultra short term fund',
 'bandhan us equity fund of fund',
 'bandhan us treasury bond 0-1 year fund of fund',
 'bandhan_asset_allocation_fund__conservative_plan',
 'bank of india arbitrage',
 'bank of india balanced',
 'bank of india bluechip fund',
 'bank of india business cycle',
 'bank of india conservative',
 'bank of india consumption',
 'bank of india credit risk fund',
 'bank of india elss',
 'bank of india flexi cap',
 'bank of india large & mid',
 'bank of india liquid fund',
 'bank of india manufacturing &',
 'bank of india mid & small cap',
 'bank of india multi asset',
 'bank of india multi cap',
 'bank of india overnight fund',
 'bank of india short term income',
 'bank of india small cap',
 'bank of india ultra short',
 'banking_and_psu_fund',
 'Baroda BNP Paribas',
 'Baroda BNP Paribas 2026 Index Fund',
 'Baroda BNP Paribas 2028 Index Fund',
 'Baroda BNP Paribas Aggressive Hybrid Fund',
 'Baroda BNP Paribas Aqua Fund of Fund',
 'Baroda BNP Paribas Arbitrage Fund',
 'Baroda BNP Paribas Business Cycle Fund',
 'Baroda BNP Paribas Corporate Bond Fund',
 'Baroda BNP Paribas Credit Risk Fund',
 'Baroda BNP Paribas Dividend Yield Fund',
 'Baroda BNP Paribas Dynamic Bond Fund',
 'Baroda BNP Paribas ELSS Tax Saver Fund',
 'Baroda BNP Paribas Equity Savings Fund',
 'Baroda BNP Paribas Flexi Cap Fund',
 'Baroda BNP Paribas Focused Fund',
 'Baroda BNP Paribas Fund',
 'Baroda BNP Paribas Gilt Fund',
 'Baroda BNP Paribas Index Fund',
 'Baroda BNP Paribas India Consumption Fund',
 'Baroda BNP Paribas Innovation Fund',
 'Baroda BNP Paribas Large & Mid Cap Fund',
 'Baroda BNP Paribas Large Cap Fund',
 'Baroda BNP Paribas Liquid Fund',
 'Baroda BNP Paribas Low Duration Fund',
 'Baroda BNP Paribas Manufacturing Fund',
 'Baroda BNP Paribas Mid Cap Fund',
 'Baroda BNP Paribas Money Market Fund',
 'Baroda BNP Paribas Multi Asset Fund',
 'Baroda BNP Paribas Multi Cap Fund',
 'Baroda BNP Paribas Nifty 50 Index Fund',
 'Baroda BNP Paribas Overnight Fund',
 'Baroda BNP Paribas Retirement Fund',
 'Baroda BNP Paribas Services Fund',
 'Baroda BNP Paribas Short Duration Fund',
 'Baroda BNP Paribas Small Cap Fund',
 'Baroda BNP Paribas Value Fund',
 'before',
 'before Credit Risk',
 'before ctor Undertakings,',
 'before debt derivatives and money',
 'before s.',
 'before y',
 'Benchmark',
 'benchmark',
 'BENCHMARK',
 'Benchmark Index',
 'benchmark:',
 'benchmark_index',
 'beta',
 'Beta',
 'Bharat Bond ETF April 2025',
 'Bharat Bond ETF APRIL 2025 ',
 'Bharat Bond ETF April 2030',
 'Bharat Bond ETF APRIL 2030 ',
 'Bharat Bond ETF April 2031',
 'Bharat Bond ETF APRIL 2031 ',
 'Bharat Bond ETF April 2032',
 'Bharat Bond ETF APRIL 2032 ',
 'Bharat Bond ETF April 2033',
 'Bharat Bond ETF APRIL 2033 ',
 'Bharat Bond ETF FOF APRIL 2032 ',
 'Bharat Bond ETF FOF APRIL 2033 ',
 'Bharat Bond FOF APRIL 2025 ',
 'Bharat Bond FOF APRIL 2030 ',
 'Bharat Bond FOF APRIL 2031 ',
 'Bloomberg Code',
 'bloomberg_code',
 'Bonus Option',
 'bse_&_nse_scrip_code',
 'bse_code',
 'bse_scrip_code',
 'business_day_of_the_month',
 'canara robeco balanced advantage fund (crbaf)',
 'canara robeco banking and psu debt fund (crbpdf)',
 'canara robeco blue chip equity fund (crbcef)',
 'canara robeco conservative hybrid fund (crchf)',
 'canara robeco consumer trends fund (crctf)',
 'canara robeco corporate bond fund (crcbf)',
 'canara robeco dynamic bond fund (crdbf)',
 'canara robeco elss tax saver (crets)',
 'canara robeco emerging equities (cree)',
 'canara robeco equity hybrid fund (crehf)',
 'canara robeco flexicap fund (crfcf)',
 'canara robeco focused equity fund (crfef)',
 'canara robeco gilt fund (crgilt)',
 'canara robeco income fund (crinc)',
 'canara robeco infrastructure (cri)',
 'canara robeco liquid fund (crl)',
 'canara robeco manufacturing fund (crmtf)',
 'canara robeco mid cap fund (crmcf)',
 'canara robeco multi cap fund (crmucf)',
 'canara robeco overnight fund (crof)',
 'canara robeco savings fund (crsf)',
 'canara robeco short duration fund (crsdf)',
 'canara robeco small cap fund (crscf)',
 'canara robeco ultra short term fund (crustf)',
 'canara robeco value fund (crvf)',
 'category',
 'Category',
 'Category of Scheme',
 'CATEGORY/TYPE',
 'category_of_the_scheme',
 'Closing AUM',
 'closing_aum',
 'co_fund_manager',
 'com',
 'commercial_paper',
 'company_name',
 'company_name_isin_y',
 'company_name_isin_yt',
 'company_name_isin_ytm_ytc',
 'composition_by_asset_(%)',
 'composition_by_asset_(%)_composition_by_industry_(%)',
 'composition_by_industry_(%)',
 'composition_by_rating_(%)_composition_by_asset_(%)',
 'conservative_hybrid_fund',
 'corporate_bond_fund',
 'corporate_debt_market_development_fund',
 'cpse etf',
 'Creation Unit Size',
 'creation_unit_size',
 'credit_risk_fund',
 'd corporate bonds).',
 'Date of Allotment',
 'DATE OF ALLOTMENT',
 'Date of allotment',
 'date of allotment',
 'date_of_allotment',
 'date_of_allotment:',
 'date_of_allotment_main_portfolio',
 'date_of_inception/allotment',
 'date_of_recategorisation',
 'days_from_the_date_of_allotment',
 'debt_&_debt_related',
 'debt_investments',
 'debt_parameter_(as_on_december_31,_2024)',
 'debt_portion',
 'Dematerialization',
 'designation',
 'details_as_on_november_30,_2024',
 'development_fund',
 'Direct',
 'Direct  Growth',
 'direct - growth',
 'Direct - Growth Plan',
 'Direct  IDCW',
 'direct - idcw',
 'Direct - IDCW Plan',
 'Direct Growth',
 'Direct Growth Option',
 'direct growth v',
 'Direct IDCW',
 'Direct IDCW (Payout)',
 'Direct IDCW Option',
 'direct idcw v',
 'direct monthly idcw',
 'Direct Plan',
 'direct plan',
 'direct plan - adhoc idcw option v',
 'Direct Plan - Annual IDCW Option',
 'direct plan - annual idcw option v',
 'direct plan - daily idcw option v',
 'direct plan - dividend option',
 'Direct Plan - Growth',
 'direct plan - growth option',
 'direct plan - growth option v',
 'Direct Plan - IDCW Monthly',
 'direct plan - idcw option',
 'direct plan - idcw option v',
 'direct plan - monthly idcw option v',
 'Direct Plan - Quarterly IDCW Option',
 'direct plan - quarterly idcw option v',
 'direct plan - weekly idcw option v',
 'Direct Plan (G)',
 'Direct Plan Annual IDCW Option',
 'direct plan daily idcw',
 'Direct Plan Daily IDCW Option',
 'Direct Plan Fortnightly IDCW Option',
 'direct plan growth',
 'Direct Plan Growth Option',
 'direct plan growth v',
 'Direct Plan IDCW',
 'Direct Plan IDCW Annual',
 'Direct Plan IDCW Fortnightly',
 'Direct Plan IDCW Option',
 'Direct Plan Monthly IDCW Option',
 'Direct Plan Quarterly IDCW Option',
 'direct plan weekly idcw',
 'Direct Plan Weekly IDCW Option',
 'direct plan-growth option v',
 'direct plan-idcw v',
 'direct quarterly idcw',
 'directly_with_the_mutual_fund:',
 'doubt_about_whether_the_product_is_suitable_f',
 'dsp 10y g-sec fund',
 'dsp aggressive hybrid fund',
 'dsp arbitrage fund',
 'dsp banking & financial services fund',
 'dsp banking & psu debt fund',
 'dsp bond fund',
 'dsp bse liquid rate etf',
 'dsp bse sensex etf',
 'dsp business cycle fund',
 'dsp corporate bond fund',
 'dsp credit risk fund',
 'dsp crisil sdl plus g-sec apr 2033 50:50 index fund',
 'dsp dynamic asset allocation fund',
 'dsp elss tax saver fund (erstwhile known as dsp tax saver fund)',
 'dsp equity opportunities fund',
 'dsp equity savings fund',
 'dsp flexi cap fund',
 'dsp floater fund',
 'dsp focus fund',
 'dsp gilt fund (erstwhile dsp government',
 'dsp healthcare fund',
 'dsp india t.i.g.e.r. fund',
 'dsp liquidity fund',
 'dsp low duration fund',
 'dsp mid cap fund',
 'dsp multi asset allocation fund',
 'dsp multicap fund',
 'dsp natural resources and new energy fund',
 'dsp nifty 1d rate liquid etf',
 'dsp nifty 50 equal weight etf',
 'dsp nifty 50 equal weight index fund',
 'dsp nifty 50 etf',
 'dsp nifty 50 index fund',
 'dsp nifty bank etf',
 'dsp nifty bank index fund',
 'dsp nifty healthcare etf',
 'dsp nifty it etf',
 'dsp nifty midcap 150 quality 50 etf',
 'dsp nifty midcap 150 quality 50 index fund',
 'dsp nifty next 50 index fund',
 'dsp nifty private bank etf',
 'dsp nifty psu bank etf',
 'dsp nifty sdl plus g-sec jun 2028 30:70 index fund',
 'dsp nifty sdl plus g-sec sep 2027 50:50 index fund',
 'dsp nifty smallcap250 quality 50 index fund',
 'dsp nifty top 10 equal weight etf',
 'dsp nifty top 10 equal weight index fund',
 'dsp quant fund',
 'dsp regular savings fund',
 'dsp savings fund',
 'dsp short term fund',
 'dsp silver etf',
 'dsp small cap fund',
 'dsp strategic bond fund',
 'dsp top 100 equity fund',
 'dsp ultra short fund',
 'dsp value fund',
 'duration',
 'duration of the',
 'duration_%_of_assets',
 'dynamic_bond_fund',
 'e',
 'Edelweiss Aggressive Hybrid Fund ',
 'Edelweiss Arbitrage Fund ',
 'Edelweiss Asean Equity Off-Shore Fund ',
 'Edelweiss Balanced Advantage Fund ',
 'Edelweiss Banking & PSU Debt Fund ',
 'Edelweiss BSE Capital Markets & Insurance ETF ',
 'Edelweiss Business Cycle Fund ',
 'Edelweiss CRISIL IBX 50:50 Gilt Plus SDL Apr 2037 Index Fund ',
 'Edelweiss CRISIL IBX 50:50 Gilt Plus SDL June 2027 Index Fund ',
 'Edelweiss CRISIL IBX 50:50 Gilt Plus SDL Sep 2028 Index Fund ',
 'Edelweiss CRISIL IBX 50:50 Gilt Plus SDL Short Duration Index Fund ',
 'Edelweiss CRISIL IBX AAA Financial Services Bond - Jan 2028 Index Fund ',
 'Edelweiss CRISIL PSU Plus SDL 50:50 Oct 2025 Index Fund ',
 'Edelweiss ELSS Tax saver Fund ',
 'Edelweiss Emerging Markets Opportunities Equity Offshore Fund ',
 'Edelweiss Equity Savings Fund ',
 'Edelweiss Europe Dynamic Equity Off-Shore Fund ',
 'Edelweiss Flexi-Cap Fund ',
 'Edelweiss Focused Fund ',
 'Edelweiss Gold and Silver ETF Fund of Fund ',
 'Edelweiss Gold ETF ',
 'Edelweiss Government Securities Fund ',
 'Edelweiss Greater China Equity Off-Shore Fund ',
 'Edelweiss Large & Mid Cap Fund ',
 'Edelweiss Large Cap Fund ',
 'Edelweiss Liquid Fund ',
 'Edelweiss Mid Cap Fund ',
 'Edelweiss Money Market Fund ',
 'Edelweiss MSCI India Domestic & World Healthcare 45 Index Fund ',
 'Edelweiss Multi Asset Allocation Fund ',
 'Edelweiss Multi Cap Fund ',
 'Edelweiss Nifty 100 Quality 30 Index Fund ',
 'Edelweiss Nifty 50 Index Fund ',
 'Edelweiss Nifty Alpha Low Volatility 30 Index Fund ',
 'Edelweiss Nifty Bank ETF ',
 'Edelweiss Nifty Large Mid Cap 250 Index Fund ',
 'Edelweiss Nifty Midcap 150 Momentum 50 Index Fund ',
 'Edelweiss Nifty Next 50 Index Fund ',
 'Edelweiss NIFTY PSU Bond Plus SDL Apr - 2026 50:50 Index Fund ',
 'Edelweiss NIFTY PSU Bond Plus SDL Apr - 2027 50:50 Index Fund ',
 'Edelweiss Nifty Smallcap 250 Index Fund ',
 'Edelweiss Nifty500 Multicap Momentum Quality 50 ETF ',
 'Edelweiss Nifty500 Multicap Momentum Quality 50 Index Fund ',
 'Edelweiss Overnight Fund ',
 'Edelweiss Recently Listed IPO Fund ',
 'Edelweiss Silver ETF ',
 'Edelweiss Small Cap Fund ',
 'Edelweiss Technology Fund ',
 'Edelweiss US Technology Equity Fund of Fund ',
 'Edelweiss US Value Equity Off-Shore Fund ',
 'elss',
 'ely high interest',
 'Entry Load',
 'ENTRY LOAD',
 'entry_load',
 'entry_load:',
 'Equity',
 'equity turnover',
 'equity_&_equity_related',
 'equity_-_etf',
 'equity_-_retirement',
 'equity_futures',
 'equity_investments',
 'equity_parameter_(as_on_december_31,_2024)',
 'equity_savings',
 'european_opportunities_fund',
 'exchange_listed',
 'exchange_symbol',
 'exchange_traded_funds',
 'existing_investors',
 'Exit Load',
 'EXIT LOAD',
 'exit_load',
 'exit_load:',
 'Expense Ratio',
 'EXPENSE RATIO',
 'Expense Ratio as on',
 'expense_ratio',
 'expense_ratio_',
 'expense_ratio_direct',
 'expense_ratio_direct_',
 'experience',
 'experience_in_managing_the_fund',
 'f_i',
 'Face Value per Unit',
 'feeder__franklin_us_opportunities_fund',
 'fixed_income_-_etf',
 'fixed_income_-_retirement',
 'flexi_cap_fund',
 'floater_fund',
 'focused_fund',
 'for_existing_investors',
 'for_franklin_india_feeder__templeton',
 'for_new_investors',
 'for_subscription_as_per_the_below_details',
 'for_the_month',
 'for_transactions_directly_with_the_fund:',
 'for_transactions_on_stock_exchanges:',
 'Formerly known as IIFL Dynamic Bond Fund',
 'Formerly known as IIFL ELSS NIFTY 50 Tax Saver Index Fund',
 'franklin asian equity fund',
 'franklin build india fund',
 'franklin india arbitrage fund',
 'franklin india balanced advantage fund',
 'franklin india banking & psu debt fund',
 'franklin india bluechip fund',
 'franklin india corporate debt fund',
 'franklin india debt hybrid fund',
 'franklin india elss tax saver fund',
 'franklin india equity advantage fund',
 'franklin india equity hybrid fund',
 'franklin india equity savings fund',
 'franklin india feeder - franklin u.s. opportunities fund',
 'franklin india feeder - templeton european opportunities fund',
 'franklin india flexi cap fund',
 'franklin india floating rate fund',
 'franklin india focused equity fund',
 'franklin india government securities fund',
 'franklin india liquid fund',
 'franklin india long duration fund',
 'franklin india medium to long duration fund',
 'franklin india money market fund',
 'franklin india multi cap fund',
 'franklin india multi-asset solution fund of funds',
 'franklin india nse nifty 50 index fund',
 'franklin india opportunities fund',
 'franklin india overnight fund',
 'franklin india pension plan',
 'franklin india prima fund',
 'franklin india smaller companies fund',
 'franklin india technology fund',
 'franklin india ultra short duration fund',
 'fund',
 'Fund Manager',
 'FUND MANAGER',
 'Fund Managers',
 'Fund of Fund',
 'Fund P/B',
 'Fund Size',
 'Fund Size Monthly Average',
 'fund_features',
 'fund_manager',
 'fund_manager:',
 'futures',
 'gilt_fund',
 'Gold - FoF',
 'government_bond',
 'Growth',
 'Growth Plan',
 'hedged_equity_exposure',
 'helios balanced advantage fund',
 'helios financial services fund',
 'helios flexi cap fund',
 'helios large & mid cap fund',
 'helios overnight fund',
 'IDCW',
 'IDCW Plan',
 'idcw**_history',
 'idcw_history',
 'inception',
 'Inception Date',
 'inception_date',
 'inception_date_01aug22',
 'inception_date_01dec2021',
 'inception_date_05mar24',
 'inception_date_06sep24',
 'inception_date_07feb19',
 'inception_date_07nov23',
 'inception_date_08may15',
 'inception_date_10mar22',
 'inception_date_10nov22',
 'inception_date_11oct22',
 'inception_date_12aug09',
 'inception_date_12dec22',
 'inception_date_13dec21',
 'inception_date_13feb14',
 'inception_date_13oct14',
 'inception_date_13sep13',
 'inception_date_14dec2022',
 'inception_date_14jun07',
 'inception_date_14oct21',
 'inception_date_14sep22',
 'inception_date_15dec2021',
 'inception_date_15feb23',
 'inception_date_18mar21',
 'inception_date_18oct22',
 'inception_date_18th_july_2006',
 'inception_date_18th_june_2024',
 'inception_date_1st_april_1995',
 'inception_date_20aug09',
 'inception_date_21nov23',
 'inception_date_21sep07',
 'inception_date_21st_november_2022',
 'inception_date_22feb18',
 'inception_date_22jul20',
 'inception_date_22july20',
 'inception_date_23jul2020',
 'inception_date_23jun23',
 'inception_date_23rd_september_2008',
 'inception_date_24jul19',
 'inception_date_24may24',
 'inception_date_25may16',
 'inception_date_25oct23',
 'inception_date_25th_june_2003',
 'inception_date_26dec07',
 'inception_date_26dec19',
 'inception_date_26oct2020',
 'inception_date_27jun08',
 'inception_date_27jun14',
 'inception_date_27th_september_2006',
 'inception_date_29jul24',
 'inception_date_29th_august_2022',
 'inception_date_2nd_june_1997',
 'inception_date_30dec08',
 'inception_date_30dec2019',
 'inception_date_30nov2022',
 'inception_date_31oct24',
 'inception_date_31st_december_1997',
 'inception_date_31st_march_2008',
 'inception_date_3feb15',
 'inception_date_3rd_december_2019',
 'inception_date_5th_march_2008',
 'income_portion',
 'index_fund',
 'india_elss_tax_saver_w.e.f._december_26,_2023.',
 'Indicative Investment Horizon',
 'Information Ratio',
 'initial__additional_investment',
 'initial_investment',
 'invesco india - invesco eqqq nasdaq-100 etf fund of fund',
 'invesco india - invesco global consumer trends fund of fund',
 'invesco india - invesco global equity income fund of fund',
 'invesco india - invesco pan european equity fund of fund',
 'invesco india aggressive hybrid fund',
 'invesco india arbitrage fund',
 'invesco india balanced advantage fund',
 'invesco india banking and psu fund',
 'invesco india contra fund',
 'invesco india corporate bond fund',
 'invesco india credit risk fund',
 'invesco india elss tax saver fund',
 'invesco india equity savings fund',
 'invesco india esg integration strategy fund',
 'invesco india financial services fund',
 'invesco india flexi cap fund',
 'invesco india focused fund',
 'invesco india gilt fund',
 'invesco india gold etf fund of fund',
 'invesco india gold exchange traded fund',
 'invesco india infrastructure fund',
 'invesco india large & mid cap fund',
 'invesco india largecap fund',
 'invesco india liquid fund',
 'invesco india low duration fund',
 'invesco india manufacturing fund',
 'invesco india medium duration fund',
 'invesco india midcap fund',
 'invesco india money market fund',
 'invesco india multi asset allocation fund',
 'invesco india multicap fund',
 'invesco india nifty 50 exchange traded fund',
 'invesco india nifty g-sec jul 2027 index fund',
 'invesco india overnight fund',
 'invesco india psu equity fund',
 'invesco india short duration fund',
 'invesco india smallcap fund',
 'invesco india technology fund',
 'invesco india ultra short duration fund',
 'Investment Objective',
 'investment_objective',
 'investor_exit_upon_subscription_exit_load_(as_a_%_of_redemption_proceeds)',
 'investors_should_consult_their_financial_advi',
 'investors_who_are_seeking_riskometer',
 'ip_of_10000_per_month)',
 'isin',
 'iti arbitrage fund',
 'iti balanced advantage fund',
 'iti banking & psu debt fund',
 'iti banking and financial services fund',
 'iti dynamic bond fund',
 'iti elss tax saver fund',
 'iti flexi cap fund',
 'iti focused equity fund',
 'iti large & mid cap fund',
 'iti large cap fund',
 'iti liquid fund',
 'iti mid cap fund',
 'iti multi cap fund',
 'iti overnight fund',
 'iti pharma and healthcare fund',
 'iti small cap fund',
 'iti ultra short duration fund',
 'iti value fund',
 'Jenson',
 'Jensons',
 'jm aggressive hybrid fund',
 'jm arbitrage fund',
 'jm dynamic bond fund',
 'jm elss tax saver fund',
 'jm flexicap fund',
 'jm focused fund',
 'jm large cap fund',
 'jm liquid fund',
 'jm low duration fund',
 'jm medium to long duration fund',
 'jm midcap fund',
 'jm overnight fund',
 'jm short duration fund',
 'jm small cap fund',
 'jm value fund',
 'kotak balanced advantage fund',
 'kotak banking & financial services fund',
 'kotak banking and psu debt fund',
 'kotak bluechip fund',
 'kotak bond fund',
 'kotak bond short term fund',
 'kotak bse housing index fund',
 'kotak bse psu index fund',
 'kotak bse sensex etf',
 'kotak business cycle fund',
 'kotak consumption fund',
 'kotak corporate bond fund',
 'kotak credit risk fund',
 'kotak crisil-ibx aaa financial services index – sep 2027 fund',
 'kotak debt hybrid fund',
 'kotak dynamic bond fund',
 'kotak elss tax saver fund',
 'kotak emerging equity fund',
 'kotak equity arbitrage fund',
 'kotak equity hybrid fund',
 'kotak equity opportunities fund',
 'kotak equity savings fund',
 'kotak esg exclusionary strategy fund',
 'kotak flexicap fund',
 'kotak floating rate fund',
 'kotak focused equity fund',
 'kotak gilt fund',
 'kotak global emerging market fund',
 'kotak global innovation fund of fund',
 'kotak gold etf',
 'kotak gold fund',
 'kotak healthcare fund',
 'kotak income plus arbitrage fof',
 'kotak india eq contra fund',
 'kotak international reit fof',
 'kotak liquid fund',
 'kotak long duration fund',
 'kotak low duration fund',
 'kotak manufacture in india fund',
 'kotak medium term fund',
 'kotak mnc fund',
 'kotak money market fund',
 'kotak multi asset allocation fund',
 'kotak multicap fund',
 'kotak nasdaq 100 fund of fund',
 'kotak nifty 100 equal weight index fund',
 'kotak nifty 100 low volatility 30 etf',
 'kotak nifty 100 low volatility 30 index fund',
 'kotak nifty 1d rate liquid etf',
 'kotak nifty 200 momentum 30 index fund',
 'kotak nifty 50 equal weight index fund',
 'kotak nifty 50 etf',
 'kotak nifty 50 index fund',
 'kotak nifty 50 value 20 etf',
 'kotak nifty aaa bond jun 2025 htm index fund',
 'kotak nifty alpha 50 etf',
 'kotak nifty financial services ex-bank index fund',
 'kotak nifty g-sec july 2033 index fund',
 'kotak nifty india consumption etf',
 'kotak nifty india tourism index fund',
 'kotak nifty midcap 150 momentum 50 index fund',
 'kotak nifty midcap 50 etf',
 'kotak nifty midcap 50 index fund',
 'kotak nifty mnc etf',
 'kotak nifty next 50 index fund',
 'kotak nifty psu bank etf',
 'kotak nifty sdl apr 2027 top 12 equal weight index fund',
 'kotak nifty sdl apr 2032 top 12 equal weight index fund',
 'kotak nifty sdl jul 2026 index fund',
 'kotak nifty sdl jul 2033 index fund',
 'kotak nifty sdl plus aaa psu bond jul 2028 60:40 index fund',
 'kotak nifty smallcap 50 index fund',
 'kotak overnight fund',
 'kotak pioneer fund',
 'kotak quant fund',
 'kotak savings fund',
 'kotak small cap fund',
 'kotak special opportunities fund',
 'kotak technology fund',
 'kotak transportation & logistics fund',
 'Large Cap Fund- An open ended e',
 'large_&_mid_cap_fund',
 'large_cap_fund',
 'Latest AUM',
 'latest_aum',
 'latest_aum#',
 'Launched',
 'liquid_fund',
 'listing_exchange',
 'load',
 'Load Structure',
 'lock_in_period',
 'Lock-in period',
 'lockin_period__minimum',
 'lockinperiod',
 'long_duration_fund',
 'long-term_performance_10,000_invested',
 'low_duration_fund',
 'lumpsum_details',
 'macaulay duration',
 'Macaulay Duration',
 'macaulay_duration',
 'macaulay_duration_',
 'Mahindra Manulife Aggressive Hybrid Fund',
 'Mahindra Manulife Arbitrage Fund',
 'Mahindra Manulife Asia Pacific REITs FOF',
 'Mahindra Manulife Balanced Advantage Fund',
 'Mahindra Manulife Business Cycle Fund',
 'Mahindra Manulife Consumption Fund',
 ...]

In [ ]:
entry_pattern = r"Entry Load\s*(Not Applicable)\s*Exit Load"
exit_pattern = r"Exit Load\s*(.*?)$"  # Capture until next "Entry Load" or end of text

# Extract matches
entry_loads = re.findall(entry_pattern, all_text, re.IGNORECASE)
exit_loads = re.findall(exit_pattern, all_text, re.IGNORECASE | re.DOTALL)
exit_loads = [re.sub(r"\s+", " ", load.strip()) for load in exit_loads]

# Combine extracted data
extracted_data = [{"Entry Load": entry, "Exit Load": exit} for entry, exit in zip(entry_loads, exit_loads)]


In [ ]:
mention_start = [
        "Date of Allotment",
        "Fund Manager",
        "Fund Size",
        "Load Structure",
        "Benchmark",
        "Minimum Additional",
        r"Minimum Redemption\s*\/\s*Switch-out Amount",
    ]
    
mention_end = mention_start[1:] + ["End_of_Data"]

# Generate regex patterns dynamically
patterns = [r"({start}\s*)(.+?)({end}|$)".format(start=start, end=end)
    for start, end in zip(mention_start, mention_end)]
final_dict = {}